In [ ]:
import requests
import time
import os
import json
from typing import List
from tqdm import tqdm

SLEEP_BETWEEN_REQUESTS = 1
SEMANTIC_PATH = '../semanticscholar_raw_data'
S2_API_KEY = os.environ.get('S2_API_KEY', '')

# paper_id = 'CorpusId:258959144'

FIELDS = ','.join([
    'paperId',
    'title',
    'venue',
    'publicationVenue',
    'year',
    'authors',
    'fieldsOfStudy',
    'publicationTypes',
    'publicationDate',
    'journal',
    'citations',
    'abstract'
])

PAYLOAD = {'fields': FIELDS, 'X-API-KEY': S2_API_KEY}

if not os.path.exists(SEMANTIC_PATH):
    os.mkdir(SEMANTIC_PATH)

def cache_request(paper_id: str) -> int:
    """
    This function checks if @paper_id has already been requested and stored in disk.
    If it has not been requested yet, it will request it and store it in disk if the status code is successful.
    It returns the status code of the request.
    """
    path = f'{SEMANTIC_PATH}/{paper_id}.json'
    if not os.path.exists(path):
        time.sleep(SLEEP_BETWEEN_REQUESTS)
        url = f'https://api.semanticscholar.org/graph/v1/paper/{paper_id}'
        r = requests.get(url, params=PAYLOAD)

        if r.status_code == 200:
            # print(f"Got paper {paper_id}")
            with open(path, 'w') as f:
                f.write(r.text)
        else:
            print(f"Fail: status code {r.status_code} paper_id {paper_id}")

        return r.status_code
    else:
        # print(f"{paper_id} already in disk")
        return 0  # "Status code" for file already in disk.

# This is the Hive paper
paper_id = '19a22753a1ef61c2bd19e97961f8ed1b5a073c2c'
cache_request(paper_id)

def visit_cited_papers() -> List[str]:
    """
    Returns a list of paper_id's that were cited by any of the papers in `SEMANTIC_PATH`.
    """
    cited = []
    for fname in os.listdir(SEMANTIC_PATH):
        with open(f'{SEMANTIC_PATH}/{fname}') as f:
            paper = json.loads(f.read())
    
        for citation in paper.get('citations', []):
            cited.append(citation['paperId'])

    return cited

visit_cited_papers()

for paper_id in tqdm(visit_cited_papers()):
    r = cache_request(paper_id)

# Docs

{'paperId': '1226d86b1333a93141b76d88e3c3c9c5c8ac1fb0', 'publicationVenue': None, 'title': 'Elastic Pipelining in an In-Memory Database Cluster', 'venue': 'SIGMOD Conference', 'year': 2016, 'fieldsOfStudy': ['Computer Science'], 'publicationTypes': ['JournalArticle', 'Book', 'Conference'], 'publicationDate': '2016-06-14', 'journal': {'name': 'Proceedings of the 2016 International Conference on Management of Data'}, 'authors': [{'authorId': '2152714702', 'name': 'Li Wang'}, {'authorId': '39337286', 'name': 'Minqi Zhou'}, {'authorId': '2109512262', 'name': 'Zhenjie Zhang'}, {'authorId': '47796371', 'name': 'Y. Yang'}, {'authorId': '145031580', 'name': 'Aoying Zhou'}, {'authorId': '1909942', 'name': 'D. Bitton'}], 'citations': [{'paperId': 'c4c193b3fca96a8b16f3dbe3247ad36389073537', 'title': 'Krypton: Real-time Serving and Analytical SQL Engine at ByteDance'}, {'paperId': 'f3e8edfb0e6d6844a8a017e8100647b6fbf63c4b', 'title': 'On inter-operator data transfers in query processing'}, {'paperId': '71bfa79c702c230b257c5fe277b2e642d01a6673', 'title': 'Pipelined XPath Query Based on Cost Optimization'}, {'paperId': '6b57fffef9323858122084c4f5cc4ffdd922eec3', 'title': 'A-DSP: An Adaptive Join Algorithm for Dynamic Data Stream on Cloud System'}, {'paperId': 'bc61dce150ff871386f85481970bfc1d1c10941c', 'title': 'Out-of-order execution of database queries'}, {'paperId': 'd0eede72be60fd031cbab0b7fe527e5dbe8808b8', 'title': 'PA-Tree: Polled-Mode Asynchronous B+ Tree for NVMe'}, {'paperId': 'fb2f9dd392b22e99ebc1c9087c41d6e1a9238f1e', 'title': 'Handling Data Skew for Aggregation in Spark SQL Using Task Stealing'}, {'paperId': 'c4d3bba752b4b15c05b3d71a9de2a73fe94ec8a2', 'title': 'Scheduling Resources to Multiple Pipelines of One Query in a Main Memory Database Cluster'}, {'paperId': '954bb881fd92ff5c5ef7b729a8b85d984c39a715', 'title': 'To pipeline or not to pipeline, that is the question'}, {'paperId': '4bd927570edc775c0ed38496491549a7bab0b68e', 'title': 'Waterwheel: Realtime Indexing and Temporal Range Query Processing over Massive Data Streams'}, {'paperId': 'f963a5bab702915567f7cc3d85dd40a77110362a', 'title': 'Parallelizing Multiple Pipelines of One Query in a Main Memory Database Cluster'}, {'paperId': 'e639762852d31f57697e1f96aef60cc3587a58b1', 'title': 'Elasticutor: Rapid Elasticity for Realtime Stateful Stream Processing'}, {'paperId': 'b79a8fb33c8209ef6c02d43335affa8e7129eabd', 'title': 'Empowering In-Memory Relational Database Engines with Native Graph Processing'}, {'paperId': 'd2189bf80b2c1bd55c80b6c189bd4c8a1464c667', 'title': 'DRS: Auto-Scaling for Real-Time Stream Analytics'}, {'paperId': 'c7f9193a0bcaa3a0601b47183a2842535c3de6df', 'title': 'DITIR: Distributed Index for High Throughput Trajectory Insertion and Real-time Temporal Range Query'}, {'paperId': '90ef13ee34dff3ed4fa3779e5b9b87afc7e30ad0', 'title': 'Adaptive Concurrent Query Execution Framework for an Analytical In-Memory Database System'}, {'paperId': '71b2f44bb489feb5e7562a43a2d0c7c4f22b5c42', 'title': 'Cost-Effective Stream Join Algorithm on Cloud System'}, {'paperId': 'c5f74dc959ba45880ebc346d08ca8fb30f874b83', 'title': 'the Proceedings of the 18th USENIX Symposium on Networked Systems Design and Implementation.'}, {'paperId': '21810bed01c0873d2568840814465717093f710e', 'title': 'HetExchange: Encapsulating heterogeneous CPU-GPU parallelism in JIT compiled engines'}, {'paperId': '9ec2f0d95acc8606323e39054dfab18fe1ca301c', 'title': 'Extending In-Memory Relational Database Engines with Native Graph Support'}]}

In this domain, authors write research papers that
can be published in the proceedings of a conference or workshop
, or in a journal. A conference/workshop is organized in terms of editions.
Each edition of a conference/workshop is held in a given city (venue) at a specific period of
time of a given year.Proceedings are published records which include all the papers presented
in an edition of a conference/workshop. Oppositely, journals do not hold joint meeting events
and, like a magazine, a journal publishes accepted papers in terms of volumes. There can be
various volumes of a journal per year.
A paper can be written by many authors, however only one of them acts as corresponding
author. A paper can be cited by another paper, meaning their content is related. A paper
can be about one or more topics, specified by means of keywords (e.g., property graph, graph
processing, data quality, etc.). A paper must also contain an abstract (i.e., a summary of its
content).

Finally, we also want to include in the graph the concept of review. When a paper is submit-
ted to a conference/workshop or a journal, the conference chair or the journal editor assigns
a set of reviewers (typically three) to each paper. Reviewers are scientists and therefore they
are relevant authors (i.e., they have published papers in relevant conferences or journals).
Obviously, the author of a certain paper cannot be reviewer of her own paper.

- [ ] Keywords
- [ ] Workshops are funky
- [ ] Reviews - How do they work?

# TODO